In [1]:
import os
import moviepy.editor as mp
import sys


In [2]:
tmp_folder = 'tmp/'
filename = 'video_youtube.mp4'#'test_meet.mp4'
context = 'relgas del juego de mesa virus' #'reunión de trabajo entre el equipo de Datatouille y el equipo de DataPlatform'
file_name_output = os.path.splitext(filename)[0]
file_name_mp3 = os.path.join(tmp_folder, file_name_output + '.mp3')
file_name_compress_mp3 = os.path.join(tmp_folder, file_name_output + '_compress.mp3')
file_name_txt = os.path.join(tmp_folder, file_name_output + '.txt')
audio_language = 'es' #'en' https://en.wikipedia.org/wiki/List_of_ISO_639_language_codes
summary_language = 'Spanish'

In [6]:
my_clip = mp.VideoFileClip(filename=filename)
my_clip.audio.write_audiofile(filename = file_name_mp3)


                                                                    
chunk:   0%|          | 35/9572 [00:49<3:45:50,  1.42s/it, now=None]

MoviePy - Writing audio in tmp/video_youtube.mp3










































                                                                    
chunk:   0%|          | 35/9572 [00:53<4:04:02,  1.54s/it, now=None]

MoviePy - Done.


In [7]:
%%time
!ffmpeg \
-i "$file_name_mp3" \
-ar 16000 \
-ac 1 \
-map 0:a: \
"$file_name_compress_mp3" \
-y



ffmpeg version 7.0.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --ena

In [8]:
file_size = os.path.getsize(file_name_compress_mp3)

# 25MB en bytes
size_limit = 25 * 1024 * 1024

file_size_mb = round(file_size / (1024 * 1024))

if file_size > size_limit:
    print(f"El fichero {filename} es más grande de 25MB ({file_size_mb}MB).")
    sys.exit("Tamaño máximo del fichero 25MB")

In [9]:
import os
from groq import Groq

client = Groq()
filename = file_name_compress_mp3

with open(filename, "rb") as file:
    transcription = client.audio.transcriptions.create(
      file=(filename, file.read()),
      model="whisper-large-v3-turbo",
      response_format="json", #verbose_json", #Set to verbose_json to receive timestamps for audio segments. Set to text to return a text response.
      prompt=context, #"Specify context or spelling",
      language=audio_language,
      temperature=0.0
    )
    print(transcription.text)


 Bienvenidos al vídeo de cómo se juega a virus. Si no habéis visto el vídeo de presentación, en la descripción os dejo un link que os llevará el vídeo. Veamos cómo se juega. La preparación del juego es muy sencilla, se cogen todas las cartas del juego, se barajan y se reparten 3 cartas a cada jugador En este caso vamos a realizar el ejemplo con 3 jugadores y colocaremos las cartas boca arriba para que las veáis El resto de cartas se quedan formando el mazo de robo con esto ya estaría preparado el juego. Se elige un jugador inicial, imaginemos este, y podríamos empezar a jugar. Vamos a explicar cómo funciona un turno. El jugador activo tiene dos opciones. Puede o jugar una de las cartas de su mano, o descartarse de cualquier cantidad de cartas que haya en su mano. Si decide, por ejemplo, jugar una carta de su mano, colocar el hueso en su cuerpo, habría realizado su acción. Al finalizar su turno debe robar tantas cartas del mazo como necesite para tener tres cartas en su mano. en este ca

In [10]:
output_filename = file_name_txt
with open(output_filename, "w") as output_file:
    output_file.write(transcription.text)

In [11]:
with open(file_name_txt, "r") as file:
    transcript = file.read()

In [12]:
from pprint import pprint
from bedrock_client import BedrockClient

MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0"
# MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"
# MODEL_ID = "anthropic.claude-3-5-sonnet-20240620-v1:0"
BedrockClient = BedrockClient()

SUMARIZATION_PROMPT=f"""You are responsible for accurately summarizing the meeting <transcript>.

<transcript>
{transcript}
</transcript>

Think step by step on how to summarize the <transcript> within the provided <sketchpad>.

In the <sketchpad>, return a list of <decision>, <action_item>, and <owner>.

Then, check that <sketchpad> items are factually consistent with the <transcript>.

Finally, return a short <summary> based on the <sketchpad>.

Write the answer in {summary_language}

Don't hallucinate. Don't make up truthful information.
"""

completion = BedrockClient.invoke_model(MODEL_ID, system_prompt="", user_prompt=SUMARIZATION_PROMPT)

response_text = completion.get("content")[0]["text"]
print(response_text)

<sketchpad>
<decision>Explicar cómo se juega al juego de cartas "Virus"</decision>
<action_item>Preparar el juego: barajar y repartir 3 cartas a cada jugador</action_item>
<owner>Jugador inicial</owner>
<action_item>Explicar las opciones de un turno: jugar una carta o descartar cartas</action_item>
<action_item>Explicar el objetivo del juego: tener 4 órganos sanos en la mesa</action_item>
<action_item>Explicar los virus y cómo infectan los órganos</action_item>
<action_item>Explicar las medicinas y cómo se pueden usar para curar o vacunar los órganos</action_item>
<action_item>Explicar los tratamientos especiales: trasplante, robo de órganos, contagio y cambio de cuerpo</action_item>
<action_item>Explicar la carta "Guantes de látex" y su efecto</action_item>
<action_item>Explicar la carta "Célula madre" y su efecto</action_item>
</sketchpad>

<summary>
En el vídeo se explica cómo se juega al juego de cartas "Virus". Se describe la preparación del juego, repartiendo 3 cartas a cada juga

In [10]:
from gtts import gTTS
import re

def test(text, language='en', slow=False):
    speech = gTTS(text=text, lang=language, slow=slow)
    speech_file = 'speech.mp3'
    speech.save(speech_file)
    # Play the audio file
    os.system('afplay ' + speech_file)

In [12]:
# match = re.search(r'<summary>(.*?)</summary>', response_text, re.DOTALL)
# if match:
#     summary_text = match.group(1).strip()
#     print(summary_text)
# test(text=summary_text,language='es')

En esta reunión se explicó el proceso paso a paso para agregar una nueva creatividad en la plataforma AdVendio. El proceso incluye iniciar sesión, seleccionar una campaña existente, agregar una nueva creatividad, configurar los detalles de la creatividad (tamaño, nombre, URL de destino) y finalmente enviar la creatividad a moderación.
